Première étape : séparer les données en deux groupes, training set et testing set avec une répartion de 80/20

In [308]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor

In [309]:
data =pd.read_csv('https://raw.githubusercontent.com/bachir151/stage/master/building_seattle.csv',on_bad_lines='skip')

In [310]:
data_model =data.drop(['Unnamed: 0'], axis=1)

In [311]:
data_model =data_model.drop(['index'], axis=1)

In [312]:
variables = ['SiteEnergyUse(kBtu)','TotalGHGEmissions','NumberofBuildings','NumberofFloors','PropertyGFATotal', 'PropertyGFAParking',
             'PropertyGFABuilding(s)', 'ENERGYSTARScore', 'age_batiment', 'LargestPropertyUseType', 'PrimaryPropertyType', 'BuildingType',
             'Ratio_buildingParSurface', 'Ratio_parkingParSurface', 'SurfaceParetage']

In [313]:
numerical_features = ['NumberofBuildings','NumberofFloors','PropertyGFATotal', 'ENERGYSTARScore', 'age_batiment']
categorical_features =['LargestPropertyUseType', 'PrimaryPropertyType', 'BuildingType']

In [320]:
data_model['PrimaryPropertyType'].value_counts()

Other                          547
Small- and Mid-Sized Office    287
Warehouse                      187
Large Office                   166
Mixed Use Property             112
Retail Store                    90
Hotel                           76
Worship Facility                71
Name: PrimaryPropertyType, dtype: int64

In [314]:
categorical_features 

['LargestPropertyUseType', 'PrimaryPropertyType', 'BuildingType']

In [315]:
X=data_model[['NumberofBuildings','NumberofFloors','PropertyGFATotal', 'ENERGYSTARScore', 'age_batiment', 'LargestPropertyUseType', 'PrimaryPropertyType', 'BuildingType']]

Y=data_model[['SiteEnergyUse(kBtu)','TotalGHGEmissions']]

In [316]:
# standardiser les données
preprocessor = make_column_transformer((RobustScaler(),numerical_features),(OneHotEncoder(handle_unknown = 'ignore'),categorical_features))
X_train, X_test, y_train, y_test = train_test_split(X, Y['TotalGHGEmissions'], test_size=0.2)
results = []

##Régression linéaire

In [317]:
model = make_pipeline(preprocessor,LinearRegression())
parameters = {
    'linearregression__fit_intercept':[True,False]
}
grid = GridSearchCV(model,parameters, cv=5)
grid.fit(X_train, y_train)
print('Méthode: LinearRegression OneHotEncoder RobustScaler')
print ("best score: ", grid.best_score_ )
print("best params: ",grid.best_params_)
y_pred = grid.predict(X_test)
print("score de la prédiction:")#, accuracy_score(y_test, y_pred)), 
print("RMSE = ",mean_absolute_error(y_test,y_pred))
print("MAE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
print("median abs err = ",median_absolute_error(y_test,y_pred))
print("R2= ",r2_score(y_test,y_pred))

results.append(['LinearRegression', grid.best_score_,mean_absolute_error(y_test,y_pred),np.sqrt(mean_squared_error(y_test,y_pred)),median_absolute_error(y_test,y_pred), grid.best_params_])

Méthode: LinearRegression OneHotEncoder RobustScaler
best score:  -0.13356936934348757
best params:  {'linearregression__fit_intercept': True}
score de la prédiction:
RMSE =  192.04817110619254
MAE =  471.2642540088576
median abs err =  93.60977143955222
R2=  0.17699923371315596


In [ ]:
model = make_pipeline(preprocessor,ElasticNet(max_iter=10000,tol=100))
n_alphas = 10
alphas = np.logspace(-5, 5, n_alphas)
l1_ratio = np.logspace(-5, 1, n_alphas)
parameters = {'elasticnet__alpha':alphas,  'elasticnet__l1_ratio':l1_ratio}
grid = GridSearchCV(model,parameters, cv=5, n_jobs = 5)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print('Méthode: ElasticNet OneHotEncoder RobustScaler')
print ("best score: ", grid.best_score_)
print("best params: ",grid.best_params_)
print("score de la prédiction:"), 
print("RMSE = ",mean_absolute_error(y_test,y_pred))
print("MAE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
print("median abs err = ",median_absolute_error(y_test,y_pred))

results.append(['ElasticNet', grid.best_score_,mean_absolute_error(y_test,y_pred),np.sqrt(mean_squared_error(y_test,y_pred)),median_absolute_error(y_test,y_pred), grid.best_params_])